In [0]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/fluent_python_notes"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/fluent_python_notes


In [0]:
import os
import imp

## 4.1 字符问题

- 一个字符串是一个字符序列，“字符”的最佳定义为 Unicode 字符
- Unicode 把字符的标识和具体字节表述作了如下区分
  - 字符的标识，即码位，在 Unicode 标准中以 4~6 个十六进制数字表示，而且加上前缀 "U+"
    - Unicode 6.3 中，约 10% 的有效码位有对应的字符
  - 字符的具体表述取决于所用的编码
    - 编码是在码位和字节序列之间转换时使用的算法
    - 在 UTF-8 编码中，A（U+0041）的码位编码成单个字节 \x41，而在 UTF-16LE 编码中编码成两个字节 \x41\x00
- 把码位转换为字节序列使用是编码，把字节序列转换为码位使用的是解码

###### 示例 4-1 编码和解码

In [0]:
s = 'café'  # s 中有 4 个 unicode 字符
len(s)

4

In [0]:
b = s.encode('utf8')  # 使用 utf-8 将 str 对象编码为 bytes 对象
b  # bytes 字面量以 b 开头

b'caf\xc3\xa9'

In [0]:
len(b)  # utf-8 中 é 码位编码成两个字节

5

In [0]:
b.decode('utf8')  # 使用 utf-8 将 bytes 对象解码为 str 对象

'café'

## 4.2 字节概要

- `bytes` 和 `bytearray` 对象的各个元素介于 `0-255`(含) 之间的整数
- 二进制序列的切片始终是同一类型的二进制序列，包括长度为 $1$ 的切片
***
- 各个字节的值可能会使用下列三种不同的方式显示
  - 可打印的 ASCII 范围内的字节（从空格到 ~ ）, 使用 ASCII 字符本身
  - 制表符，换行符、回车符和 \ 对应的字符，使用转义序列 \t、 \n、 \r 和 \\
  - 其它字符的值，使用十六进制的转义序列 （\x00 是空字节）


###### 示例4-2 包含 5 个字节的 `bytes` 和 `bytearray` 对象

In [0]:
cafe = bytes('café', encoding='utf-8')
cafe

b'caf\xc3\xa9'

In [0]:
cafe[0]  # 返回一个整数  即返回一个元素

99

In [0]:
cafe[:1]  # 返回一个 bytes 对象 即返回一个序列

b'c'

In [0]:
cafe_arr = bytearray(cafe)
cafe_arr

bytearray(b'caf\xc3\xa9')

In [0]:
cafe_arr[-1:]

bytearray(b'\xa9')

##### 处理二进制序列的方法
- 除了格式化方法( `format` 和 `format_map` ） 和几个处理 Unicode 数据的方法之外， `str` 类型的其
它方法均支持 `bytes` 和 `bytesarray` 类型， 即可以使用大部分字符串方法处理二进制序列
- 二进制序列有一个 `str` 没有的类方法，名为 `fromhex`，其作用是解析十六进制数字对，构建二进制序列

In [0]:
bytes.fromhex('31 4B CE A9')

b'1K\xce\xa9'

##### 构建 `bytes` 和 `bytearray` 实例的方法

- 一个 `str` 对象和一个 `encoding` 关键字参数
- 一个可迭代对象，提供 0 ~ 255 之间的数值
- 一个实现了缓冲协议的对象（如 `bytes`, `bytearray`, `memoryview`, `array.array`）, 此时会将源对象中的字节序列复制到新建的二进制序列中

###### 示例 4-3 使用数组中的原始数据初始化 `bytes` 对象

In [0]:
import array
numbers = array.array('h', [-2, -1, 0, 1, 2])  # h 指代短整数(16 位) 数组
octets = bytes(numbers)
octets

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'

### 结构体和内存视图

- `struct` 模块提供了一些函数，其会将字节序列转换成不同类型字段组成的元组
- `memoryview` 用于共享内存，可以直接访问其它二进制序列、打包的数组和缓冲中的数据切片，而无需复制字节序列

###### 示例 4-4 使用 `memoryview` 和 `struct` 查看一个 GIF 图像的首部

In [0]:
import struct
fmt = '<3s3sHH'  # < 表示小字节序， 3s3s 是两个 3 字节序列， HH 是两个十六进制的整数
with open('ch4/filter.gif', 'rb') as fp:
  img = memoryview(fp.read())
headers = img[:10]
bytes(headers)

b'GIF89a,\x01,\x01'

In [0]:
struct.unpack(fmt, headers)  # 拆包 memoryview 对象，得到一个元组，包含类型、版本、宽度和高度

(b'GIF', b'89a', 300, 300)

In [0]:
del headers
del img  # 删除引用，释放内存

## 4.3 基本的编解码器

- 常见的编码
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200313192118.png width=800>

###### 示例 4-5 使用 3 个编解码器编码字符串 "El NiñO"

In [0]:
for codec in ['latin_1', 'utf_8', 'utf_16']:
  print(codec, 'El Niño'.encode(codec), sep='\t\t')

latin_1		b'El Ni\xf1o'
utf_8		b'El Ni\xc3\xb1o'
utf_16		b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


## 4.4 了解编解码问题

### 4.4.1 处理 `UnicodeEncodeError`

- 把文本转换为字节序列时，如果目标编码中没有定义某个字符，则会抛出 `UnicodeEncodeError` 异常，除非把 
`errors` 参数传给编码方法或函数，对错误进行特殊处理

###### 示例 4-6 编码成字节序列：成功和错误处理

In [0]:
city = 'São Paulo'
city.encode('utf_8')

b'S\xc3\xa3o Paulo'

In [0]:
city.encode('utf_16')

b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'

In [0]:
city.encode('iso8859_1')

b'S\xe3o Paulo'

In [0]:
city.encode('cp437')  # cp437 编码无法识别编码 ã ， 会抛出 UnicodeEncodeError

UnicodeEncodeError: ignored

In [0]:
city.encode('cp437', errors='ignore')  # ignore 会忽略错误，不进行任何处理

b'So Paulo'

In [0]:
city.encode('cp437', errors='replace')  # replace 会把无法编码的字符替换为 '?'

b'S?o Paulo'

In [0]:
city.encode('cp437', errors='xmlcharrefreplace')  # 将无法编码的字符替换为 XML 实体

b'S&#227;o Paulo'

### 4.4.2 处理 `UnicodeDecodeError`

- 将二进制序列转换为文本时，如果遇到选定编码无法转换的字节序列时，会抛出 `UnicodeDecodeError` 
- 许多陈旧的 8 位编码 -- 如 `cp1252`, `iso8859_1`, `koi8_r` -- 能解码任何字节序列流而不抛出错误
  - 但有可能得到无用的输出

###### 示例 4-7 把字节序列解码成字符串：成功和错误处理

In [0]:
octets = b'Montr\xe9al'
octets.decode('cp1252')

'Montréal'

In [0]:
octets.decode('iso8859_7')  # 此编码用于编码希腊文，其无法正常解释 \xe9 字节，但是却没有抛出错误

'Montrιal'

In [0]:
octets.decode('koi8_r')  # 此编码用于编码希腊文，基会钭 \xe9 翻译为其它字节

'MontrИal'

In [0]:
octets.decode('utf_8')  # utf-8 检测到 \xe9 不是有效的 UTF-8 字符串，抛出 UnicodeDecodeError

UnicodeDecodeError: ignored

In [0]:
octets.decode('utf_8', errors='replace')  # 使用 replace 处理错误，则 \xe9 被替换为 �(码位是 U+FFFD)，其是官方指定的 REPLACEMENT CHARACTER(替换字符)，表示未知字符

'Montr�al'

### 4.4.3 使用预期之外编码加载模块时抛出 SyntaxError 

- Python3 默认使用 UTF-8 编码源码，如果加载的 .py 模块中含有 UTF-8 之外的数据，而且没有声明编码，则
会提示 `SynataxError`
- 为了修正这个问题，可以在文件开头加上 `coding` 注释
  - ```python3
    # coding: cp1252
  ```

#### Python3 源码中可以使用非 ASCII 名称

In [0]:
ε = 10 ** -6

In [0]:
ε

1e-06

### 4.4.4 找出字节序列的编码

- 文本的字节编码必须被指定，但是可以通过试探和分析找出编码
- 统一字符编码侦测包 Chardet

In [0]:
import glob
from chardet.universaldetector import UniversalDetector

detector = UniversalDetector()
for filename in glob.glob('*.ipynb'):  # 找出当前路径下符合给定条件的所有文件
    print(filename.ljust(60))
    detector.reset()
    with open(filename, 'rb') as f:
      for line in f.readlines():
        detector.feed(line)
        if detector.done: break
    detector.close()
    print(detector.result)


第 4 章：文本和字节序列.ipynb                                         
{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}
第 1 章：Python 数据类型.ipynb                                     
{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}
第 2 章：序列构成的数组.ipynb                                         
{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}
第 3 章: 字典和集合.ipynb                                          
{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


### 4.4.5 BOM: 有用的鬼符

- UTF-16 编码的序列开头有几个额外的字节，如下例：

In [0]:
u16 = 'El Niño'.encode('utf_16')
u16

b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'

- BOM 即是指 `\xff\xfe`，即字节序标记(byte-order mark)，其指明编码使用的是 Intel CPU 的小字节序
  - 实质上是特殊的不可见字符 ZERO WIDTH NO-BREAK SPACE (`U+FEFF`)
***
- 字母 `E` 的码位是 U+0045, 在字节偏移的第 2 位和第 3 位的编码为 69 和 0

In [0]:
list(u16)

[255, 254, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]

***
- UTF-16 有两个变种。如果使用这两个变种，则不会生成 BOM
  - UTF-16LE， 显式指明使用小字节序
  - UTF-16BE, 显式指明使用大字节序


In [0]:
u16le = 'El Niño'.encode('utf_16le')
list(u16le)

[69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]

In [0]:
u16be = 'El Niño'.encode('utf_16be')
list(u16be)

[0, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111]

***
- 按照标准，如果文件使用 UTF-16 编码，而且没有 BOM，则应该假定其使用的是 UTF-16BE 编码
  - Intel X86 架构采用提小字节序，因此有很多不带 BOM 的小字节序 UTF-16 编码
- 与字节序有关的问题，只对一个字占多个字节的编码有关，而对于 UTF-8 以一个字节为单位处理的编码来说，没有影响
  - 因为 UTF-8 不管使用哪种设备，生成的字节序列始终一致
  - UTF-8 不需要 BOM, 但有些文件仍然会在文件中添加 BOM
    - 由于 `U+FEFF` 的 utf-8 编码为 `\xef\xbb\xbf`，所以文本以这三个字节开头，则可能是带有 BOM 的 utf-8 文件
    - Python 不会因为文件以 `\xbf\xbb\xbf` 开头就认为其是 utf-8 编码的 

## 4.5 处理文本文件

##### unicode 三明治

- 处理文本的最佳实践是 unicode 三明治
  - 尽早的把输入的字节序列解码为字符串
  - 在程序的业务逻辑中，只处理字符串对象
  - 对于输出来说，则是尽量晚的将输出编码为字节序列
- 示意图
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200314211452.png width=800>


##### 依赖默认的编码可能出现错误

###### 示例 4-9 写入与打开编码不一致导致的问题

In [0]:
open('ch4/cafe.txt', 'w', encoding='utf-8').write('café')

4

In [0]:
open('ch4/cafe.txt', encoding='cp1252').read()

'cafÃ©'

###### 示例 4-10 仔细分析示例 4-9

In [0]:
fp = open('ch4/cafe.txt', 'w', encoding='utf-8')  # 默认情况下， open 函数采用文本模式，返回一个 TextIOWrapper 对角
fp

<_io.TextIOWrapper name='ch4/cafe.txt' mode='w' encoding='utf-8'>

In [0]:
fp.write('café')  # 会返回写入的 Unicode 字符数

4

In [0]:
fp.close()

In [0]:
os.stat('ch4/cafe.txt').st_size  # 返回文件的字节数为 5，因为在 utf-8 编码中， é 占据两个字节， 为 0xc3 和 0xa9

5

In [0]:
fp2 = open('ch4/cafe.txt', encoding='cp1252')
fp2

<_io.TextIOWrapper name='ch4/cafe.txt' mode='r' encoding='cp1252'>

In [0]:
fp2.encoding  # 返回 TextIOWrapper 对象

'cp1252'

In [0]:
fp2.read()  # 在cp1252 编码中， 0xc3 对应Ã， 0xa9 对应 ©

'cafÃ©'

In [0]:
fp2.close()

In [0]:
fp3 = open('ch4/cafe.txt', encoding='utf_8')  # 按正确的编码读到文件
fp3

<_io.TextIOWrapper name='ch4/cafe.txt' mode='r' encoding='utf_8'>

In [0]:
fp3.read()

'café'

In [0]:
fp3.close()

In [0]:
fp4 = open('ch4/cafe.txt', 'rb')  # 以二进制的形式读取文件
fp4

<_io.BufferedReader name='ch4/cafe.txt'>

In [0]:
fp4.read()

b'caf\xc3\xa9'

In [0]:
fp4.close()

### 编码默认值: 一团糟

###### 示例 4-11 探索编码默认值

- 有几个设置对 Python 的编码默认值有影响，可以通过下述文件获取

In [0]:
%%writefile ch4/default_encodings.py
import sys, locale

expressions = """
    locale.getpreferredencoding()
    type(my_file)
    my_file.encoding
    sys.stdout.isatty()
    sys.stdout.encoding
    sys.stdin.isatty()
    sys.stdin.encoding
    sys.stderr.isatty()
    sys.stderr.encoding
    sys.getdefaultencoding()
    sys.getfilesystemencoding()
"""

my_file = open('ch4/dummy', 'w')

for expression in expressions.split():
  value = eval(expression)
  print(expression.rjust(30), '->', repr(value))

Overwriting ch4/default_encodings.py


In [0]:
!python3 ch4/default_encodings.py

 locale.getpreferredencoding() -> 'UTF-8'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'UTF-8'
           sys.stdout.isatty() -> True
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> True
            sys.stdin.encoding -> 'UTF-8'
           sys.stderr.isatty() -> True
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


- 在 GNU/Linux 和 OS X 中，输出的结果始终是 utf-8 编码

##### 默认编码值

- 如果打开文件时没有指定 `encoding` 参数，默认值由 `locale.getpreferredencoding()` 提供
- 如果输入/输出重定向到文件，则由 `locale.getpreferredencoding()` 定义
- `sys.getfilesystemencoding()` 用于编码文件名（不是文件内容）
  - 将字符串参数作为文件名传递给 open() 函数时，会使用它
- windows 下的编码比较复杂，而且各个编码值也会受到用户设置的影响，因此最好不要使用默认编码

## 4.6 为了正确比较而规范化 Unicode 字符串

***
- “café”这个词可以使用两种方式构成，分别有 4 个和 5 个码位，但是结果完全一样

In [0]:
s1 = 'café'
s2 = 'cafe\u0301'
s1, s2

('café', 'café')

In [0]:
len(s1), len(s2)

(4, 5)

In [0]:
s1 == s2

False

- U+0301 是 COMBINING ACUTE ACCENT，加在“e”后面得到“é”。在 Unicode 标准中，'é' 和 'e\u0301' 这样的序列叫“标准等价物”（canonical equivalent）
- 应用程序应该把它们视作相同的字符。但是，Python 看到的是不同的码位序列，因此判定二者不相等
***
- 这个问题的解决方案是使用 unicodedata.normalize 函数提供的 Unicode 规范化。这个函数的第一个参数是这 4 个字符串中的一个：'NFC'、'NFD'、'NFKC' 和 'NFKD'

###### NFC NFD

***
- NFC（Normalization Form C）使用最少的码位构成等价的字符串
- NFD 把组合字符分解成基字符和单独的组合字符

In [0]:
from unicodedata import normalize
s1 = 'café'
s2 = 'cafe\u0301'
len(s1), len(s2)

(4, 5)

In [0]:
len(normalize('NFC', s1)), len(normalize('NFC', s2))

(4, 4)

In [0]:
len(normalize('NFD', s1)), len(normalize('NFD', s2))

(5, 5)

In [0]:
normalize('NFC', s1) == normalize('NFC', s2)

True

In [0]:
normalize('NFD', s1) == normalize('NFD', s2)

True

- 西方键盘通常能输出组合字符，因此用户输入的文本默认是 NFC 形式。不过，安全起见，保存文本之前，最好使用 `normalize('NFC',user_text)` 清洗字符串
***
- 使用 NFC 时，有些单字符会被规范成另一个单字符。例如，电阻的单位欧姆（Ω）会被规范成希腊字母大写的欧米加。这两个字符在视觉上是一样的，但是比较时并不相等，因此要规范化，防止出现意外

In [0]:
from unicodedata import name
ohm = '\u2126'
name(ohm)

'OHM SIGN'

In [0]:
ohm_c = normalize('NFC', ohm)
name(ohm_c)

'GREEK CAPITAL LETTER OMEGA'

In [0]:
ohm == ohm_c

False

In [0]:
normalize('NFC', ohm) == normalize('NFC', ohm_c)

True

###### NFKC 与 NFKD

- NFKC 和 NFKD 的首字母缩略词中，字母 K 表示“compatibility”（兼容性）。这两种是较严格的规范化形式，对“兼
容字符”有影响
- 在 NFKC 和 NFKD 形式中，各个兼容字符会被替换成一个或多个“兼容分解”字符
  - 这样会有格式损失，但仍是“首选”表述——理想情况下，格式化是外部标记的职责，不应该由 Unicode 处理
  - 二分之一 '½'（U+00BD）经过兼容分解后得到的是三个字符序列 '1/2'；
  - 微符号 'μ'（U+00B5）经过兼容分解后得到的是小写字母 'μ'（U+03BC)

In [0]:
half = '½'
normalize('NFKC', half)

'1⁄2'

In [0]:
four_squared= '4²'
normalize('NFKC', four_squared)

'42'

In [0]:
micro = 'µ'
micro_kc = normalize('NFKC', micro)
micro, micro_kc

('µ', 'μ')

In [0]:
ord(micro), ord(micro_kc)

(181, 956)

In [0]:
name(micro), name(micro_kc)

('MICRO SIGN', 'GREEK SMALL LETTER MU')

- NFKC 和 NFKD 只能在特殊情况中使用，例如搜索和索引，而不能用于持久存储，因为这两种转换会导致数据损失

### 4.6.1 大小写折叠

- 大小写折叠其实就是把所有文本变成小写，再做些其他转换
- 这个功能由 `str.casefold()` 方法（Python 3.3 新增）支持
- 对于只包含 latin1 字符的字符串 `s`，`s.casefold()` 得到的结果与 `s.lower()` 一样，唯有两个例外：
  - 微符号 'μ' 会变成小写的希腊字母“μ”（在多数字体中二者看起来一样）；
  - 德语 Eszett（“sharp s”，ß）会变成“ss”。

In [0]:
micro = 'µ'
name(micro)

'MICRO SIGN'

In [0]:
micro_cf = micro.casefold()
name(micro_cf)

'GREEK SMALL LETTER MU'

In [0]:
micro, micro_cf

('µ', 'μ')

In [0]:
eszett = 'ß'
name(eszett)

'LATIN SMALL LETTER SHARP S'

In [0]:
eszett_cf = eszett.casefold()
eszett, eszett_cf

('ß', 'ss')

### 4.6.2 规范化文本匹配实用函数

In [0]:
%%writefile ch4/normeq.py
"""Utility functions for normalized Unicode string comparison."""
from unicodedata import normalize

def nfc_equal(str1, str2):
  """Using Normal Form C, case sensitive"""
  return normalize('NFC', str1) == normalize('NFC', str2)

def fold_equal(str1, str2):
  """Using Normal Form C with case folding"""
  return normalize('NFC', str1).casefold() == normalize('NFC', str2).casefold()


Overwriting ch4/normeq.py


In [0]:
from ch4.normeq import nfc_equal, fold_equal

In [0]:
s1 = 'café'
s2 = 'cafe\u0301'
s1 == s2

False

In [0]:
nfc_equal(s1, s2)

True

In [0]:
nfc_equal('A', 'a')

False

In [0]:
s3 = 'Straße'
s4 = 'Strasse'

s3 == s4

False

In [0]:
nfc_equal(s3, s4)

False

In [0]:
fold_equal(s3, s4)

True

In [0]:
fold_equal(s1, s2)

True

In [0]:
fold_equal('A', 'a')

True

### 4.6.3 极端 "规范化"：去掉变音符号

- 在搜索中，经常忽略变音符号；除了搜索，去掉变音符号还能让 URL 更易于阅读

###### 示例 4-14 去掉全部组合记号的函数

In [0]:
%%writefile ch4/sanitize.py
import unicodedata
import string


def shave_marks(txt):
  """去掉全部变音符号"""
  norm_txt = unicodedata.normalize('NFD', txt)
  shaved = ''.join(c for c in norm_txt if not unicodedata.combining(c))
  return unicodedata.normalize('NFC', shaved)

Overwriting ch4/sanitize.py


In [0]:
import ch4.sanitize 
from ch4.sanitize import shave_marks

In [0]:
order = '“Herr Voß: • ½ cup of OEtker™ caffè latte • bowl of açaí.”'
shave_marks(order)

'“Herr Voß: • ½ cup of OEtker™ caffe latte • bowl of acai.”'

In [0]:
Greek = 'Ζέφυρος, Zéfiro'
shave_marks(Greek)

'Ζεφυρος, Zefiro'

***
- 通常，去掉变音符号是为了把拉丁文本变成纯粹的 ASCII，但是 shave_marks 函数还会修改非拉丁字符（如希腊字母），而只去掉重音符并不能把它们变成 ASCII 字符
- 因此，我们应该分析各个基字符，仅当字符在拉丁字母表中时才删除附加的记号

In [0]:
%%writefile -a ch4/sanitize.py



def shave_marks_latin(txt):
  """将拉打基字符中所有的变音符号删除"""
  norm_txt = unicodedata.normalize('NFD', txt)
  latin_base = False
  keepers = []
  for c in norm_txt:
    if unicodedata.combining(c) and latin_base:
      continue  # 怱略拉丁基字符上的变音符号
    keepers.append(c)
    if not unicodedata.combining(c):  # 如果不是组合字符，那就是新的基字符
      latin_base = c in string.ascii_letters
  shaved = ''.join(keepers)
  return unicodedata.normalize('NFC', shaved)

Appending to ch4/sanitize.py


In [0]:
imp.reload(ch4.sanitize)
from ch4.sanitize import shave_marks_latin

In [0]:
order = '“Herr Voß: • ½ cup of OEtker™ caffè latte • bowl of açaí.”'
shave_marks_latin(order)

'“Herr Voß: • ½ cup of OEtker™ caffe latte • bowl of acai.”'

In [0]:
Greek = 'Ζέφυρος, Zéfiro'
shave_marks_latin(Greek)  # “έ” 不会被替换

'Ζέφυρος, Zefiro'

###### 示例 4-17 将一些西文印刷字符转换为 ASCII 字符

In [0]:
%%writefile -a ch4/sanitize.py



single_map = str.maketrans("""‚ƒ„†ˆ‹‘’“”•–—˜›""",
                """'f"*^<''""---~>""")

multi_map = str.maketrans({
    '€': '<euro>',
    '…': '...',
    'Œ': 'OE',
    '™': '(TM)',
    'œ': 'oe',
    '‰': '<per mille>',
    '‡': '**'
})

multi_map.update(single_map)


def dewinize(txt):
  """将 Win1252 符号替换成 ASCII 字符或序列, 替换弯引号、项目符号和™（商标符号）"""
  return txt.translate(multi_map)


def asciize(txt):
  """调用 dewinize 函数，然后去掉变音符号, 替换 'ß'"""
  no_marks = shave_marks_latin(dewinize(txt))
  no_marks = no_marks.replace('ß', 'ss')
  return unicodedata.normalize('NFKC', no_marks)

Appending to ch4/sanitize.py


In [0]:
imp.reload(ch4.sanitize)
from ch4.sanitize import  dewinize, asciize

In [0]:
order = '“Herr Voß: • ½ cup of OEtker™ caffè latte • bowl of açaí.”'
dewinize(order)

'"Herr Voß: - ½ cup of OEtker(TM) caffè latte - bowl of açaí."'

In [0]:
asciize(order)

'"Herr Voss: - 1⁄2 cup of OEtker(TM) caffe latte - bowl of acai."'

## 4.7 Unicode 文本排序

- Python 比较任何类型的序列时，会一一比较序列里的各个元素。对字符串来说，比较的是码位
  - 在比较非 ASCII 字符时，可能得不到预期的效果

###### 示例 4-19 使用 `locale.strxfrm` 函数作为排序键

In [0]:
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted(fruits)  # 排序时，重音符号和下加符对排序没有影响，但是 python 按照码位比较，得不到预期的效果
         # 正确的排序顺序应该是 ['açaí', 'acerola', 'atemoia', 'cajá', 'caju']

['acerola', 'atemoia', 'açaí', 'caju', 'cajá']

### 使用 Unicode 排序算法排序

- [PyUCA 库](https://pypi.python.org/pypi/pyuca/) 是Unicode 排序算法（Unicode Collation Algorithm，UCA）的纯 Python 实现
- 如果想定制排序方式，可以把自定义的排序表路径传给 `Collator()` 构造方法
  - PyUCA 默认使用项目自带的 [allkeys.txt](https://github.com/jtauber/pyuca)
    - 是 Unicode 6.3.0的 [“Default Unicode Collation ElementTable”](http://www.unicode.org/Public/UCA/6.3.0/allkeys.txt)的副本。


###### 示例 4-20 使用 `pyuca.Collator.sort_key` 方法

In [0]:
!pip install pyuca

     |████████████████████████████████| 1.5MB 2.8MB/s 


In [0]:
import pyuca
coll = pyuca.Collator()
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted_fruits = sorted(fruits, key=coll.sort_key)
sorted_fruits

['açaí', 'acerola', 'atemoia', 'cajá', 'caju']

## 4.8 Unicode 数据库

- Unicode 标准提供了一个完整的数据库（许多格式化的文本文件），不仅包括码位与字符名称之间的映射，还有各个字符的元数据，以及字符之间的关系。
  - Unicode 数据库记录了字符是否可以打印、是不是字母、是不是数字，或者是不是其他数值符号
    - 字符串的 `isidentifier`、`isprintable`、`isdecimal` 和 `isnumeric` 等方法就是靠这些信息作判断的
  - `str.casefold`  方法也用到了 Unicode 表中的信息

###### 示例 4-2 Unicode 数据库中数值字符的元数据示例

In [0]:
%%writefile ch4/numerics_demo.py
import unicodedata
import re

re_digit = re.compile(r'\d')

sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'

for char in sample:
  print('U+%04x' % ord(char),  # U+0000 格式的码位
     char.center(6),  # 在长度为 6 的字符串中居中显示
     're_dig' if re_digit.match(char) else '-',
     'isdig' if char.isdigit() else '-',
     'isnum' if char.isnumeric() else '-', 
     format(unicodedata.numeric(char), '5.2f'),  # 使用长度为 5，小数点保留 2 位的形式来显示字符串
     unicodedata.name(char),  # Unicode 标准中字符的名称
     sep='\t')

Overwriting ch4/numerics_demo.py


In [0]:
!python3 ch4/numerics_demo.py

U+0031	  1   	re_dig	isdig	isnum	 1.00	DIGIT ONE
U+00bc	  ¼   	-	-	isnum	 0.25	VULGAR FRACTION ONE QUARTER
U+00b2	  ²   	-	isdig	isnum	 2.00	SUPERSCRIPT TWO
U+0969	  ३   	re_dig	isdig	isnum	 3.00	DEVANAGARI DIGIT THREE
U+136b	  ፫   	-	isdig	isnum	 3.00	ETHIOPIC DIGIT THREE
U+216b	  Ⅻ   	-	-	isnum	12.00	ROMAN NUMERAL TWELVE
U+2466	  ⑦   	-	isdig	isnum	 7.00	CIRCLED DIGIT SEVEN
U+2480	  ⒀   	-	-	isnum	13.00	PARENTHESIZED NUMBER THIRTEEN
U+3285	  ㊅   	-	-	isnum	 6.00	CIRCLED IDEOGRAPH SIX


- 正则表达式 `r'\d'` 能匹配数字“1”和梵文数字 3，但是不能匹配 `isdigit` 方法判断为数字的其他字符
- `re` 模块对 Unicode 的支 持并不充分。PyPI 中有个新开发的 `regex` 模块，它的最终目的是取代 `re` 模块，以提供更好的 Unicode 支持

## 4.9 支持字符串和字节序列的双模式 API

- 标准库中的一些函数能接受字符串或字节序列为参数，然后根据类型展现不同的行为
  - `re` 和 `os` 模块中就有这样的函数。

### 4.9.1 正则表达式中的字符串和字节序列

###### 示例 4-22 字符串正则表达式和字节序列正则表达式的行为

In [0]:
%%writefile ch4/ramanujan.py
import re

re_numbers_str = re.compile(r'\d+')  # 字符串类型
re_words_str = re.compile(r'\w+')  # 字符串类型
re_numbers_bytes = re.compile(rb'\d+')  # 字节类型
re_words_bytes = re.compile(rb'\w+')  #　字节类型

text_str = ("Ramanujan saw \u0be7\u0bed\u0be8\u0bef"  # 要搜索的 Unicode 文本，包括 1729 的泰米尔数字（逻辑行直到右括号才结束
        " as 1729 = 1³ + 12³ = 9³ + 10³.")  # 这个字符串在编译时与前一个拼接起来

text_bytes = text_str.encode('utf_8')  # 字节序列只能用字节序列正则表达式搜索

print('Text', repr(text_str), sep='\n  ')  
print('Numbers')
print('  str  :', re_numbers_str.findall(text_str))  # 字符串模式 r'\d+' 能匹配泰米尔数字和 ASCII 数字
print('  bytes:', re_numbers_bytes.findall(text_bytes))  # 字节序列模式 rb'\d+' 只能匹配 ASCII 字节中的数字
print('Words')
print('  str  :', re_words_str.findall(text_str))  # 字符串模式 r'\w+' 能匹配字母、上标、泰米尔数字和 ASCII 数字
print('  bytes:', re_words_bytes.findall(text_bytes))  # 字节序列模式 rb'\w+' 只能匹配 ASCII 字节中的字母和数字


Overwriting ch4/ramanujan.py


In [0]:
!python3 ch4/ramanujan.py

Text
  'Ramanujan saw ௧௭௨௯ as 1729 = 1³ + 12³ = 9³ + 10³.'
Numbers
  str  : ['௧௭௨௯', '1729', '1', '12', '9', '10']
  bytes: [b'1729', b'1', b'12', b'9', b'10']
Words
  str  : ['Ramanujan', 'saw', '௧௭௨௯', 'as', '1729', '1³', '12³', '9³', '10³']
  bytes: [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'12', b'9', b'10']


- 字符串正则表达式有个 `re.ASCII` 标志，它让 `\w、\W、\b、\B、\d、\D、\s 和 \S` 只匹配 ASCII 字符

### 4.9.2 `os` 函数中的字符串和字节序列

- GNU/Linux 内核不理解 Unicode，，对任何合理的编码方案来说，在文件名中使用字节序列都是无效的，无法解码成字符串
- 为了规避这个问题，`os` 模块中的所有函数、文件名或路径名参数既能使用字符串，也能使用字节序列
  - 如果这样的函数使用字符串参数调用，该参数会使用 `sys.getfilesystemencoding()` 得到的编解码器自动编码
  - 然后操作系统会使用相同的编解码器解码
  - 这种行为，与 Unicode 三明治最佳实践一致
- 可以把字节序列参数传给 `os` 模块中的函数，得到字节序列返回值
  - 这一特性可以处理任何文件名或路径名，不管里面有多少鬼符

###### 示例 4-23 把字符串和字节序列传参数传给 `listdir` 函数得到的结果

In [0]:
!mkdir ch4/4_9_2
!touch ch4/4_9_2/abc.txt
!touch ch4/4_9_2/digits-of-π.txt

mkdir: cannot create directory ‘ch4/4_9_2’: File exists


In [0]:
os.listdir('ch4/4_9_2')

['abc.txt', 'digits-of-π.txt']

In [0]:
os.listdir(b'ch4/4_9_2')

[b'abc.txt', b'digits-of-\xcf\x80.txt']

###### os 模块中特殊的编码和解码函数

- `fsencode(filename)`
  - 如果 `filename` 是 `str` 类型（此外还可能是 `bytes` 类型），使用 `sys.getfilesystemencoding()` 返回的编解码器把 `filename` 编码成字节序列；否则，返回未经修改的 `filename` 字节序列
- `fsdecode(filename)`
  - 如果 `filename` 是 `bytes` 类型（此外还可能是 `str` 类型），使用 `sys.getfilesystemencoding()` 返回的编解码器把 `filename` 解码成字符串；否则，返回未经修改的 `filename` 字符串

###### 示例 4-24 使用 surrogateescape 错误处理方式

- Python 3.1 引入的 `surrogateescape` 编解码器错误处理方式是处理意外字节序列或未知编码的一种方式
  - 说明参见 [“PEP 383 — Non-decodable Bytes in System Character Interfaces”](https://www.python.org/dev/peps/pep-0383/)
  - 这种处理方式会把每个无法解码的字节替换成 Unicode 中 U+DC00 到 U+DCFF 之间的码位
  - Unicode 标准把这些码位称 为“Low Surrogate Area”，这些码位是保留的，没有分配字符，供应用程序内部使用
  - 编码时，这些码位会转换成被替换的字节值


In [0]:
os.listdir('ch4/4_9_2')

['abc.txt', 'digits-of-π.txt']

In [0]:
os.listdir(b'ch4/4_9_2')

[b'abc.txt', b'digits-of-\xcf\x80.txt']

In [0]:
pi_name_bytes = os.listdir(b'ch4/4_9_2')[1]
pi_name_str = pi_name_bytes.decode('ascii', 'surrogateescape')
pi_name_str

'digits-of-\udccf\udc80.txt'

In [0]:
pi_name_str.encode('ascii', 'surrogateescape')

b'digits-of-\xcf\x80.txt'